<a href="https://colab.research.google.com/github/aboelela924/Deep-learning-with-TF2-and-Keras/blob/master/Deep_learning_with_TF2_and_Keras_chapter_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Text Generation</h1>

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=dbea7ca6d74b8758558677ffc2b7c35f5a3024c381650bbf3a541bb8f823f887
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import numpy as np 
import os 
import re 
import shutil 
import tensorflow as tf
import wget
import zipfile

In [ ]:
DATA_DIR = "/content/data"
CHECKPOINT_DIR = "/content/data/checkpoints"
if not os.path.isdir(DATA_DIR):
    os.mkdir(DATA_DIR)
    os.mkdir(CHECKPOINT_DIR)

In [ ]:
def download_and_read(urls):
    texts = []
    for url in urls:
        data = wget.download(url, DATA_DIR)
        text = open(data, "r").read()
        text = text.replace("\ufeff", "")
        text = text.replace("\n", " ")
        text = re.sub("\s+", " ", text)
        texts.extend(text)
    return texts     

In [ ]:
texts = download_and_read([
"http://www.gutenberg.org/cache/epub/28885/pg28885.txt",
"https://www.gutenberg.org/files/12/12-0.txt"
])

In [ ]:
vocab = sorted(set(texts))
print("Vocabulary size: {:d}".format(len(vocab)))

char2idx = {c:i for i, c in enumerate(vocab)}
idx2char = {i:c for c, i in char2idx.items()}

Vocabulary size: 90


In [ ]:
print(char2idx)
print(idx2char)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, 'A': 29, 'B': 30, 'C': 31, 'D': 32, 'E': 33, 'F': 34, 'G': 35, 'H': 36, 'I': 37, 'J': 38, 'K': 39, 'L': 40, 'M': 41, 'N': 42, 'O': 43, 'P': 44, 'Q': 45, 'R': 46, 'S': 47, 'T': 48, 'U': 49, 'V': 50, 'W': 51, 'X': 52, 'Y': 53, 'Z': 54, '[': 55, ']': 56, '_': 57, 'a': 58, 'b': 59, 'c': 60, 'd': 61, 'e': 62, 'f': 63, 'g': 64, 'h': 65, 'i': 66, 'j': 67, 'k': 68, 'l': 69, 'm': 70, 'n': 71, 'o': 72, 'p': 73, 'q': 74, 'r': 75, 's': 76, 't': 77, 'u': 78, 'v': 79, 'w': 80, 'x': 81, 'y': 82, 'z': 83, '·': 84, 'ù': 85, '‘': 86, '’': 87, '“': 88, '”': 89}
{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 2

In [ ]:
texts_as_int = [char2idx[c] for c in texts]

data = tf.data.Dataset.from_tensor_slices(texts_as_int)
seq_length = 100
sequences = data.batch(seq_length+1, drop_remainder=True)

def split_train_labels(seq):
    train = seq[0:-1]
    labels = seq[1:]
    return train, labels

In [ ]:
sequences = sequences.map(split_train_labels)

In [ ]:
batch_size = 64
steps_per_epoch = ( len(texts) // seq_length ) // batch_size

In [ ]:
dataset = sequences.shuffle(10000).batch(batch_size, drop_remainder=True).repeat()

In [ ]:
class CharGenModel(tf.keras.Model):
    def __init__(self, vocab_size, num_timesteps, embedding_dim,
                 rnn_output_dim, **kwargs):
        
        super(CharGenModel, self).__init__(**kwargs)

        self.embedd = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.rnn_layer = tf.keras.layers.GRU(
            embedding_dim, 
            stateful=True, 
            return_sequences=True)
        self.dense_layer = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        x = self.embedd(x)
        x = self.rnn_layer(x)
        x = self.dense_layer(x)
        return x

In [ ]:
vocab_size = len(vocab)
num_timesteps = 100
embedding_dim = 256
rnn_output_dim = 1024
batch_size = 64

character_generator = CharGenModel(vocab_size, num_timesteps, embedding_dim, rnn_output_dim)

In [ ]:
character_generator.build(input_shape=(batch_size, seq_length))
character_generator.summary()

Model: "char_gen_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  23040     
_________________________________________________________________
gru (GRU)                    multiple                  394752    
_________________________________________________________________
dense (Dense)                multiple                  23130     
Total params: 440,922
Trainable params: 440,922
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, predictions):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions, from_logits=True)

character_generator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                            loss=loss)

In [ ]:
def generate_text(model, prefix_string, char2idx, 
                  idx2char, output_length, randomness_prop = 1):
    prefix_int = [char2idx[c] for c in prefix_string]
    prefix_int = tf.expand_dims(prefix_int, 0)
    model.reset_states()
    out = []
    for i in range(output_length):
        predictions = model(prefix_int)
        predictions = tf.squeeze(predictions, 0) / randomness_prop
        pred_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        prefix_int = tf.expand_dims([pred_id], 0)
        out.append(idx2char[pred_id])
    return prefix_string + "".join(out)

In [ ]:
num_epochs = 50

for i in range(num_epochs//10):
    checkpoint_path = "/content/data/model_after_{:d}_epochs.ckpt".format(i+1*10)
    callbacks = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    save_weights_only=True,
                                                    verbose=1)
    character_generator.fit_generator(dataset, steps_per_epoch=steps_per_epoch,
                                      epochs=10)
    
    character_generator.save_weights(checkpoint_path)
    gen_model = CharGenModel(vocab_size, seq_length, embedding_dim, rnn_output_dim)
    gen_model.load_weights(checkpoint_path).expect_partial()
    gen_model.build(input_shape=(1, seq_length))
    print(generate_text(gen_model, "Alice ", char2idx, idx2char, 100))

Epoch 1/10
54/54 [==============================] - 1s 23ms/step - loss: 1.0028
Epoch 2/10
54/54 [==============================] - 1s 21ms/step - loss: 0.9964
Epoch 3/10
54/54 [==============================] - 1s 21ms/step - loss: 0.9903
Epoch 4/10
54/54 [==============================] - 1s 22ms/step - loss: 0.9825
Epoch 5/10
54/54 [==============================] - 1s 22ms/step - loss: 0.9777
Epoch 6/10
54/54 [==============================] - 1s 22ms/step - loss: 0.9692
Epoch 7/10
54/54 [==============================] - 1s 21ms/step - loss: 0.9646
Epoch 8/10
54/54 [==============================] - 1s 22ms/step - loss: 0.9601
Epoch 9/10
54/54 [==============================] - 1s 21ms/step - loss: 0.9506
Epoch 10/10
54/54 [==============================] - 1s 22ms/step - loss: 0.9507
Alice never do,’ said the Knight seem to a bug, down in a complying, but the only knew Liture the world wa
Epoch 1/10
54/54 [==============================] - 1s 22ms/step - loss: 0.9416
Epoch 2/10
5

<h1>Sentiment Analysis</h1>

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=f184614ca17c9c5a12c4aed3cb41c13b9fc0b603f0f81c9fa1628f7408c4a211
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [61]:
import numpy as np 
import os 
import shutil
import tensorflow as tf
import wget 
import zipfile

from sklearn.metrics import accuracy_score, confusion_matrix

In [62]:
DATA_DIR = "/content/data"
CHECKPOINT_DIR = "/content/data/checkpoints"
if not os.path.isdir(DATA_DIR):
    os.mkdir(DATA_DIR)
    os.mkdir(CHECKPOINT_DIR)

In [63]:
def download_and_read(url):
    sentences = []
    labels = []
    data = wget.download(url, DATA_DIR)
    with zipfile.ZipFile(data, "r") as ref:
        ref.extractall("/content/data")  
    file_path = DATA_DIR + "/" + data.split("/")[-1].split(".")[0]
    if "(" in file_path:
        file_path = file_path.split(" (")[0]
    
    for text_file in os.listdir(file_path):
        if text_file.endswith("_labelled.txt"):
            with open(file_path+"/"+text_file, "r") as reader:
                for line in reader:
                    line = line.replace("\n", "")
                    sentence, label = line.split("\t")
                    sentences.append(sentence)
                    labels.append(int(label))
    return sentences, labels

In [64]:
sentences, labels = download_and_read("https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment labelled sentences.zip")

In [65]:
tokenizer  = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
word2idx = tokenizer.word_index
print("Vocab size: {:d}".format(len(word2idx)))
ids2word = {value: key for key, value in word2idx.items()}

vocab_size = len(word2idx)

Vocab size: 5271


In [66]:
sentences_length = np.array([ len(s.split()) for s in sentences])
print([ (p, np.percentile(sentences_length, p)) for p in [75, 80, 85, 90, 95, 99, 100] ])

[(75, 16.0), (80, 18.0), (85, 20.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


In [67]:
max_len = 64
sentences_as_int = tokenizer.texts_to_sequences(sentences)
sentences_as_int = tf.keras.preprocessing.sequence.pad_sequences(sentences_as_int, maxlen=max_len)
labels_as_int = np.array(labels)

dataset = tf.data.Dataset.from_tensor_slices((sentences_as_int, labels_as_int))

In [68]:
dataset = dataset.shuffle(10000)

test_size = len(sentences) // 3
val_size = ( len(sentences) - test_size ) // 10

test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

batch_size = 64

test_dataset = test_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
train_dataset = train_dataset.batch(batch_size)

In [69]:
class SentimentAnalysisModel(tf.keras.Model):
    def __init__(self,vocab_size, max_len, **kwargs):
        super(SentimentAnalysisModel, self).__init__(**kwargs)

        self.embedd = tf.keras.layers.Embedding(vocab_size, max_len)
        self.rnn = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(max_len)
        )
        self.fc1 = tf.keras.layers.Dense(64, activation="relu")
        self.out = tf.keras.layers.Dense(1, activation="sigmoid")

    def call(self, x):
        x = self.embedd(x)
        x = self.rnn(x)
        x = self.fc1(x)
        x = self.out(x)
        return x

In [73]:
model = SentimentAnalysisModel(vocab_size=vocab_size+1,max_len=max_len)
model.build(input_shape=(batch_size, max_len))
model.summary()

Model: "sentiment_analysis_model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  337408    
_________________________________________________________________
bidirectional_6 (Bidirection multiple                  66048     
_________________________________________________________________
dense_12 (Dense)             multiple                  8256      
_________________________________________________________________
dense_13 (Dense)             multiple                  65        
Total params: 411,777
Trainable params: 411,777
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.compile(loss="binary_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"])

In [76]:
best_model_file = os.path.join(CHECKPOINT_DIR, "best_model.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file, 
                                                save_best_only=True,
                                                save_weights_only=True,
                                                 verbose=1)
EPOCHS = 10
history = model.fit(train_dataset, epochs=EPOCHS, 
                    validation_data=val_dataset, 
                    callbacks=[checkpoint])

Epoch 1/10
25/29 [========================>.....] - ETA: 0s - loss: 0.0762 - accuracy: 0.9775
Epoch 00001: val_loss improved from inf to 0.04226, saving model to /content/data/checkpoints/best_model.h5
29/29 [==============================] - 0s 14ms/step - loss: 0.0745 - accuracy: 0.9778 - val_loss: 0.0423 - val_accuracy: 0.9800
Epoch 2/10
27/29 [==========================>...] - ETA: 0s - loss: 0.0340 - accuracy: 0.9913
Epoch 00002: val_loss improved from 0.04226 to 0.02379, saving model to /content/data/checkpoints/best_model.h5
29/29 [==============================] - 0s 14ms/step - loss: 0.0351 - accuracy: 0.9906 - val_loss: 0.0238 - val_accuracy: 0.9950
Epoch 3/10
27/29 [==========================>...] - ETA: 0s - loss: 0.0239 - accuracy: 0.9954
Epoch 00003: val_loss improved from 0.02379 to 0.01395, saving model to /content/data/checkpoints/best_model.h5
29/29 [==============================] - 0s 14ms/step - loss: 0.0239 - accuracy: 0.9956 - val_loss: 0.0140 - val_accuracy: 0.9

In [79]:
best_model = SentimentAnalysisModel(vocab_size+1, max_len)
best_model.build(input_shape=(batch_size, max_len))
best_model.load_weights(best_model_file)
best_model.compile(
loss="binary_crossentropy",
optimizer="adam",
metrics=["accuracy"]
)

In [80]:
test_loss, test_acc = best_model.evaluate(test_dataset)
print("test loss: {:.3f}, test accuracy: {:.3f}".format(test_loss, test_acc))

16/16 [==============================] - 0s 4ms/step - loss: 0.0119 - accuracy: 0.9990
test loss: 0.012, test accuracy: 0.999


<h1>POS tagging</h1>

In [102]:
import nltk

In [103]:
nltk.download("treebank")

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [104]:
import tensorflow as tf
import numpy as np 
import shutil
import os

In [105]:
DATA_DIR = "/content/data"
CHECKPOINT_DIR = "/content/data/checkpoints"
DATASETS_DIR = "/content/data/datasets"
if not os.path.isdir(DATA_DIR):
    os.mkdir(DATA_DIR)
    os.mkdir(CHECKPOINT_DIR)
    os.mkdir(DATASETS_DIR)

In [106]:
def download_and_read(datasetdir):
    sent_filename = os.path.join(datasetdir, "treebank-sents.txt")
    poss_filename = os.path.join(datasetdir, "treebank-poss.txt")

    fsents = open(sent_filename, "w")
    fposs = open(poss_filename, "w")

    sentences = nltk.corpus.treebank.tagged_sents()

    for i, sentence in enumerate(sentences):
        fsents.write(" ".join([ w for w, p in sentence ]) +"\n")
        fposs.write(" ".join([ p for w, p in sentence ]) + "\n")

    fsents.close()
    fposs.close()

    sents = []
    posses = []

    with open(sent_filename, "r") as fsents:
        for idx, line in enumerate(fsents):
            sents.append(line.strip())
    with open(poss_filename, "r") as fposs:
        for idx, line in enumerate(fposs):
            posses.append(line.strip())  

    return sents, posses  

In [107]:
def tokenize_and_buid_vocab(texts, vocab_size=None, lower=True):
    if vocab_size is None:
        tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=lower)
    else:
        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size+1, lower=lower, oov_token="UNK")
    
    tokenizer.fit_on_texts(texts)
    if vocab_size is not None:
        tokenizer.word_index = {word:index for word, index in tokenizer.word_index.items() if index <= vocab_size+1}

    word2idx = tokenizer.word_index
    idx2word = {index: word for word, index in word2idx.items()}

    return word2idx, idx2word, tokenizer

In [108]:
sentences, poss = download_and_read(DATASETS_DIR)
assert(len(sentences) == len(poss))
print("# of records: {:d}".format(len(sentences)))

# of records: 3914


In [109]:
sentences_vocab_size = 9000
poss_vocab_size = 38

word2idx_sents, idx2word_sents, tokenizer_sents = tokenize_and_buid_vocab(sentences, vocab_size=sentences_vocab_size)
word2idx_poss, idx2word_poss, tokenizer_poss = tokenize_and_buid_vocab(poss, vocab_size=poss_vocab_size, lower=False)

In [110]:
idx2word_sents[0], idx2word_poss[0] = "PAD", "PAD"

In [111]:
source_vocab_size = len(word2idx_sents)
target_vocab_size = len(word2idx_poss)
print("vocab sizes (source): {:d}, (target): {:d}".format(source_vocab_size, target_vocab_size))

vocab sizes (source): 9001, (target): 39


In [112]:
sequences_length = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(sequences_length, p)) for p in [75, 80, 85, 90, 95, 99, 100]])

[(75, 33.0), (80, 35.0), (85, 38.0), (90, 41.0), (95, 47.0), (99, 58.0), (100, 271.0)]


In [113]:
max_len = 271

sents_to_ints = tokenizer_sents.texts_to_sequences(sentences)
sents_to_ints = tf.keras.preprocessing.sequence.pad_sequences(sents_to_ints, maxlen=max_len, padding="post")

poss_to_ints = tokenizer_poss.texts_to_sequences(poss)
poss_to_ints = tf.keras.preprocessing.sequence.pad_sequences(poss_to_ints, maxlen=max_len, padding="post")

poss_to_cat_ints = []
for p in poss_to_ints:
    poss_to_cat_ints.append(tf.keras.utils.to_categorical(p, num_classes=poss_vocab_size+1, dtype="int32"))
poss_to_cat_ints = tf.keras.preprocessing.sequence.pad_sequences(poss_to_cat_ints, maxlen=max_len, padding="post")

dataset = tf.data.Dataset.from_tensor_slices((sents_to_ints, poss_to_cat_ints))

In [114]:
dataset = dataset.shuffle(10000)

test_dataset_length = len(sentences) // 3
val_dataset_length = (len(sentences) - test_dataset_length) // 10

test_dataset = dataset.take(test_dataset_length)
val_dataset = dataset.skip(test_dataset_length).take(val_dataset_length)
train_dataset = dataset.skip(test_dataset_length+val_dataset_length)

In [115]:
batch_size = 128
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [116]:
def masked_accuracy():
    def masked_accuracy_fn(ytrue, ypred):
        ytrue = tf.keras.backend.argmax(ytrue, axis=-1)
        ypred = tf.keras.backend.argmax(ypred, axis=-1)
 
        mask = tf.keras.backend.cast(
            tf.keras.backend.not_equal(ypred, 0), tf.int32)
        matches = tf.keras.backend.cast(
            tf.keras.backend.equal(ytrue, ypred), tf.int32) * mask
        numer = tf.keras.backend.sum(matches)
        denom = tf.keras.backend.maximum(tf.keras.backend.sum(mask), 1)
        accuracy =  numer / denom
        return accuracy

    return masked_accuracy_fn

In [123]:
class POSTaggingModel(tf.keras.Model):
    def __init__(self, source_vocab_size, traget_vocab_size, 
                 max_len, embedding_dim, rnn_out_dim, **kwargs):
        super(POSTaggingModel, self).__init__(**kwargs)

        self.embedding = tf.keras.layers.Embedding(source_vocab_size, 
                                                   embedding_dim, 
                                                   input_length=max_len)
        self.dropout = tf.keras.layers.SpatialDropout1D(0.2)
        self.rnn = tf.keras.layers.Bidirectional(
            tf.keras.layers.GRU(rnn_out_dim, return_sequences=True)
        )
        self.fc1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size))
        self.activation = tf.keras.layers.Activation("softmax")

    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.rnn(x)
        x = self.fc1(x)
        x = self.activation(x)
        return x

In [124]:
embedding_dim = 128
rnn_output_dim = 256

In [125]:
model = POSTaggingModel(source_vocab_size, target_vocab_size, max_len, 
                        embedding_dim, rnn_output_dim)
model.build(input_shape=(batch_size, max_len))

In [126]:
model.summary()

Model: "pos_tagging_model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     multiple                  1152128   
_________________________________________________________________
spatial_dropout1d_12 (Spatia multiple                  0         
_________________________________________________________________
bidirectional_12 (Bidirectio multiple                  592896    
_________________________________________________________________
time_distributed_12 (TimeDis multiple                  20007     
_________________________________________________________________
activation_12 (Activation)   multiple                  0         
Total params: 1,765,031
Trainable params: 1,765,031
Non-trainable params: 0
_________________________________________________________________


In [127]:
model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy", masked_accuracy()])

In [128]:
EPOCHS = 50


best_model_file = os.path.join(CHECKPOINT_DIR, "best_model.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                                    best_model_file,
                                    save_weights_only=True,
                                    save_best_only=True)
history = model.fit(train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset)

Epoch 1/50
19/19 [==============================] - 2s 127ms/step - loss: 1.4451 - accuracy: 0.8668 - masked_accuracy_fn: 0.0026 - val_loss: 0.3257 - val_accuracy: 0.9128 - val_masked_accuracy_fn: 0.0000e+00
Epoch 2/50
19/19 [==============================] - 2s 97ms/step - loss: 0.3231 - accuracy: 0.9184 - masked_accuracy_fn: 0.0803 - val_loss: 0.3231 - val_accuracy: 0.9220 - val_masked_accuracy_fn: 0.1253
Epoch 3/50
19/19 [==============================] - 2s 96ms/step - loss: 0.3147 - accuracy: 0.9225 - masked_accuracy_fn: 0.1906 - val_loss: 0.3184 - val_accuracy: 0.9148 - val_masked_accuracy_fn: 0.1889
Epoch 4/50
19/19 [==============================] - 2s 97ms/step - loss: 0.3015 - accuracy: 0.9165 - masked_accuracy_fn: 0.1234 - val_loss: 0.2867 - val_accuracy: 0.9178 - val_masked_accuracy_fn: 0.1301
Epoch 5/50
19/19 [==============================] - 2s 97ms/step - loss: 0.2814 - accuracy: 0.9176 - masked_accuracy_fn: 0.0917 - val_loss: 0.2740 - val_accuracy: 0.9218 - val_masked_